In [16]:
from pandas import DataFrame
from pandas import read_csv
df = read_csv('cs-training.csv', header=0, index_col=0)
df.fillna(0, inplace=True)
df['NumberOfTimeTotalDelay'] = df['NumberOfTime30-59DaysPastDueNotWorse'] + df['NumberOfTimes90DaysLate'] + df['NumberOfTime60-89DaysPastDueNotWorse']
df['isOld'] =df['age']>70
df.to_csv('train_full.csv')

df = read_csv('cs-test.csv', header=0, index_col=0)
df.fillna(0, inplace=True)
df['NumberOfTimeTotalDelay'] = df['NumberOfTime30-59DaysPastDueNotWorse'] + df['NumberOfTimes90DaysLate'] + df['NumberOfTime60-89DaysPastDueNotWorse']
df['isOld'] =df['age']>70
df.to_csv('test_full.csv')

import bigml.api
from bigml.api import BigML
api = BigML('MADWORM33', '0ee9369c9bb45c83f82024310ecea14943307e56', project='project/5d94a344eba31d460c000235')

source_train = api.create_source('train_full.csv')
origin_dataset = api.create_dataset(source_train)
source_test = api.create_source('test_full.csv')
test_dataset = api.create_dataset(source_test)

train_dataset = api.create_dataset(
    origin_dataset, {"name": "Training_split_Jupyter",
                     "sample_rate": 0.8, "seed": "my seed"})
val_dataset = api.create_dataset(
    origin_dataset, {"name": "Validation_split_Jupyter",
                     "sample_rate": 0.8, "seed": "my seed",
                     "out_of_bag": True})

ensemble = api.create_ensemble(train_dataset,{"name": "ensemble_Jupyter",
     "objective_field": "SeriousDlqin2yrs"})

evaluation = api.create_evaluation(ensemble, val_dataset)

batch_prediction = api.create_batch_prediction(ensemble, test_dataset, {
    "name": "ensemble_prediction_Jupyter", "all_fields": True,
    "header": True,
    "confidence": True,
    "probabilities": True})
api.ok(batch_prediction)

api.download_batch_prediction(batch_prediction,
    filename='ensemble_prediction_Jupyter.csv')

df = read_csv('ensemble_prediction_Jupyter.csv', header=0, index_col=0)



,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,NumberOfTimeTotalDelay,isOld,SeriousDlqin2yrs.1,confidence,0 probability,1 probability
field1,,,,,,,,,,,,,,,,,
1,0,0.885519,43,0,0.177513,5700,4,0,0,0,0,0,False,0,0.91260,0.93191,0.06809
2,0,0.463295,57,0,0.527237,9141,15,0,4,0,2,0,False,0,0.94690,0.96063,0.03937
3,0,0.043275,59,0,0.687648,5083,12,0,1,0,2,0,False,0,0.98514,0.98980,0.01020
4,0,0.280308,38,1,0.925961,3200,7,0,2,0,0,1,False,0,0.90449,0.92340,0.07660
5,0,1.000000,27,0,0.019917,3865,4,0,0,0,1,0,False,0,0.88931,0.91175,0.08825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101499,0,0.282653,24,0,0.068522,1400,5,0,0,0,0,0,False,0,0.97444,0.98279,0.01721
101500,0,0.922156,36,3,0.934217,7615,8,0,2,0,4,3,False,0,0.63507,0.67322,0.32678
101501,0,0.081596,70,0,836.000000,0,3,0,0,0,0,0,False,0,0.99470,0.99673,0.00327


In [18]:
df2 =df [['1 probability']]

In [19]:
df2

,1 probability
field1,
1,0.06809
2,0.03937
3,0.01020
4,0.07660
5,0.08825
...,...
101499,0.01721
101500,0.32678
101501,0.00327


In [20]:
df2.index.names = ['Id']
df2.rename(columns={"1 probability": "Probability"},inplace=True)
df2

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,Probability
Id,
1,0.06809
2,0.03937
3,0.01020
4,0.07660
5,0.08825
...,...
101499,0.01721
101500,0.32678
101501,0.00327


In [21]:
df2.to_csv('prediction_kaggle.csv')